In [1]:
%load_ext autoreload

%autoreload 2
%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")

In [2]:
from DFTStructureGenerator import DFThandle, xtb_process, mol_manipulation, logfile_process, Tool, gendes
import glob, os
from rdkit import Chem
import numpy as np
from tqdm import tqdm
import pandas as pd
from morfeus import BuriedVolume
import pickle
np.random.seed(0)

In [3]:
OPT_METHOD = "opt freq b3lyp/def2svpp em=gd3bj nosymm"

In [4]:
data_dir = "Data/DFT"
row_csv = "Data_clear.csv"
target_csv = "Data_clear_with_sites.csv"

mol_xtb_file = os.path.join(data_dir, 'Mol_xtb')
mol_dir = os.path.join(data_dir, 'Mols')
dft_dir = os.path.join(data_dir, 'GS_OPT')
cu_mol_dir = os.path.join(data_dir, 'newmols')
cu_dft_dir = os.path.join(data_dir, 'newGS')

for file_ in [mol_xtb_file, mol_dir, dft_dir, cu_mol_dir, cu_dft_dir]:
    if not os.path.exists(file_):
        os.makedirs(file_)

# 1. DFT Optimization

## Input Generation

In [41]:
smiles_csv = pd.read_csv(row_csv)
smiles_csv.loc[smiles_csv["Type"] == "Ligand_Other"]

,Index,Smiles,Type
138,1140,N[C@H]1c2ccccc2C[C@H]1[O-],Ligand_Other
139,1141,[O-]C(c1ccccc1)(c1ccccc1)[C@H]1CCCN1,Ligand_Other
140,1142,N[C@@H](c1ccccc1)[C@H]([O-])c1ccccc1,Ligand_Other
141,1143,N[C@@H]1CCCC[C@H]1[O-],Ligand_Other
142,1144,N[C@@H](C[O-])Cc1ccccc1,Ligand_Other
143,1145,[O-]C[C@@H]1CCCN1Cc1ccccc1,Ligand_Other
144,1146,N[C@H](C[O-])Cc1ccccc1,Ligand_Other
145,1147,[O-]C[C@@H]1CCCN1,Ligand_Other
146,1148,CC(C)(C)[C@@H](N)C[O-],Ligand_Other
147,1149,CC(C)[C@H](N)C[O-],Ligand_Other


In [8]:
smiles_csv = pd.read_csv(row_csv)
smiles = smiles_csv["Smiles"].values
smiles = [Chem.MolToSmiles(Chem.MolFromSmiles(each)) for each in smiles]
smiles_csv["Smiles"] = smiles
smiles_csv.to_csv(row_csv, index=False)

In [19]:
smiles_csv = pd.read_csv(row_csv)
for id, row in smiles_csv.iterrows():
    smiles = row['Smiles']
    smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles))
    row['Smiles'] = smiles
    idx = row["Index"]
    if os.path.exists(f"{mol_dir}/{idx:05}.mol"):
        continue
    mol = mol_manipulation.smiles2mol(smiles)
    Chem.AllChem.UFFOptimizeMolecule(mol)
    Chem.MolToMolFile(mol, f"{mol_dir}/{idx:05}.mol")
smiles_csv.to_csv(row_csv, index=False)

In [22]:
DFThandle.Single_Xtb(data_dir, row_csv)
xtb_process.shift_to_parra(mol_xtb_file)

In [23]:
DFThandle.smiles_DFT_calc(mol_xtb_file, mol_dir, dft_dir, method = OPT_METHOD, conf_limit = 10, SpinMultiplicity=1) 

In [25]:
DFThandle.error_improve(data_dir, mol_dir, "GS_OPT")

## Summarize Reaction Sites

In [9]:
smiles_csv = pd.read_csv(row_csv)
smiles_csv['Sites'] = {}
for row_id, row in smiles_csv.iterrows():
    binol_smiles = row["Smiles"]
    binol_mol = Chem.MolFromSmiles(binol_smiles)
    if row["Type"] == "Binol":
        subset = binol_mol.GetSubstructMatches(Chem.MolFromSmarts("[OH]c1ccccc1-c1ccccc1[OH]"))
        subset = subset[0]
        smiles_csv['Sites'][row_id] = f"{subset[0]} {subset[-1]}"
    elif row["Type"] in ["Ligand_Box", "Ligand_Other"]:
        mol = binol_mol
        Chem.Kekulize(mol, clearAromaticFlags=True)
        sites = set()
        subset = mol.GetSubstructMatches(Chem.MolFromSmarts("N~*~*~N"))
        if len(subset) == 0:
            subset = mol.GetSubstructMatches(Chem.MolFromSmarts("N~*~*~*~N"))
        if len(subset) == 0:
            subset = mol.GetSubstructMatches(Chem.MolFromSmarts("N~*~*~*~O"))
        if len(subset) == 0:
            subset = mol.GetSubstructMatches(Chem.MolFromSmarts("N~*~*~*~P"))
        if len(subset) == 0:
            subset = mol.GetSubstructMatches(Chem.MolFromSmarts("N~*~*~*~*~N"))
        if len(subset) == 0:
            subset = mol.GetSubstructMatches(Chem.MolFromSmarts("N~*~*~*~*~*~N"))
        for subset_ in subset:
            sites.add(subset_[0])
            sites.add(subset_[-1])
        smiles_csv['Sites'][row_id] = " ".join(str(site) for site in sites)
smiles_csv.to_csv(target_csv, index=False)


## Get DFT Result

In [11]:
smiles_csv = pd.read_csv(target_csv)
smiles_csv['conf_id'] = {}
smiles_csv['G/Hatree'] = {}
for row_id, row in smiles_csv.iterrows():
    ligand_idx = row["Index"]
    mol = Chem.MolFromMolFile(os.path.join(mol_dir, f"{ligand_idx:05}.mol"), removeHs=False)
    log_files = glob.glob(os.path.join(dft_dir, f"{ligand_idx:05}*.log"))
    engs = []
    conf_ids = []
    for log_file in log_files:
        conf_id = int(log_file.split("_")[-1].split(".")[0])
        log = logfile_process.Logfile(log_file)
        eng = log.all_engs[0] + log.all_engs[-1]
        engs.append(eng)
        conf_ids.append(conf_id)
    min_idx = np.argmin(engs)
    min_conf_id = conf_ids[min_idx]
    min_eng = engs[min_idx]
    smiles_csv["conf_id"][row_id] = min_conf_id
    smiles_csv['G/Hatree'][row_id] = min_eng
smiles_csv.to_csv(target_csv, index=False)

# 2. DFT Optimization With Cu2+

In [12]:
DFThandle.generate_ligand_cu_guesses(target_csv, mol_dir, dft_dir, cu_mol_dir, cu_dft_dir, M_L_DIST=1.96, OPT_METHOD=OPT_METHOD)
DFThandle.generate_binol_cu_guesses(target_csv, mol_dir, dft_dir, cu_mol_dir, cu_dft_dir, M_L_DIST=1.96, OPT_METHOD=OPT_METHOD)

[11:00:35] Molecule does not have explicit Hs. Consider calling AddHs()
[11:00:35] Molecule does not have explicit Hs. Consider calling AddHs()
[11:00:35] Molecule does not have explicit Hs. Consider calling AddHs()


In [ ]:
DFThandle.error_improve(data_dir, cu_mol_dir, "newGS")

# 3. Descriptor Calculation

## Normal DFT Descriptors

In [5]:
smiles_csv = pd.read_csv(target_csv)
qm_dict = {}
for row_id, row in tqdm(smiles_csv.iterrows()):
    des = []
    ligand_idx = row["Index"]
    ligand_smiles = row["Smiles"]
    ligand_type = row["Type"]
    ligand_conf_id = row["conf_id"]
    ligand_G = row['G/Hatree']
    sites = [int(each) for each in row['Sites'].split()]
    mol = Chem.MolFromMolFile(os.path.join(mol_dir, f"{ligand_idx:05}.mol"), removeHs=False)
    log = logfile_process.Logfile(os.path.join(dft_dir, f"{ligand_idx:05}_r_{ligand_conf_id:04}.log"))
    newlog = logfile_process.Logfile(os.path.join(cu_dft_dir, f"{ligand_idx:05}.log"))
    # new_G = newlog.all_engs[0] + newlog.all_engs[-1]
    # des += [new_G - ligand_G +1639.93]
    # log = newlog
    positions = log.running_positions[-1]
    symbol_lists = log.symbol_list
    if ligand_type == "Binol":
        subset = mol.GetSubstructMatches(Chem.MolFromSmarts("[OH]c1ccccc1-c1ccccc1[OH]"))[0]
        assert subset[0] == sites[0] and subset[-1] == sites[-1]
        sites = np.array(subset)[[0,1,6,7,-2,-1]]
        des += Tool.get_max_min_bond(positions, [[[sites[0], sites[1]], [sites[4], sites[5]]], [[sites[1], sites[2]], [sites[3], sites[4]]], [[sites[2], sites[3]], [sites[2], sites[3]]]])[:-1]
        des += Tool.get_max_min_angle(positions, [[[sites[0], sites[1], sites[2]], [sites[3], sites[4], sites[5]]], [[sites[1], sites[2], sites[3]], [sites[2], sites[3], sites[4]]]])
        des += [Tool.get_torsion_(positions[sites[1]], positions[sites[2]], positions[sites[3]], positions[sites[4]])]
        new_subset = mol.GetSubstructMatches(Chem.MolFromSmarts("[OH]c1ccc2c(cccc2)c1c3c4c(cccc4)ccc3[OH]"))
        if len(new_subset) > 0:
            new_subset = new_subset[0]
            target_idx = [[new_subset[7], new_subset[16]], [new_subset[8], new_subset[15]], [new_subset[3], new_subset[18]], [new_subset[2], new_subset[19]]]
        else:
            new_subset = subset
            target_idx = [[new_subset[5], new_subset[8]], [new_subset[4], new_subset[9]], [new_subset[3], new_subset[10]], [new_subset[2], new_subset[11]]]
        neighbor_sites = [[[neighbor.GetIdx() for neighbor in mol.GetAtomWithIdx(site).GetNeighbors() if neighbor.GetIdx() not in new_subset][0] for site in target_idx_] for target_idx_ in target_idx]
        BVs = [[BuriedVolume(symbol_lists, positions, each_neighbor + 1, include_hs=1, radius=4, z_axis_atoms=[each_target_idx + 1], excluded_atoms=[each_target_idx + 1]) for each_neighbor, each_target_idx in zip(neighbor_sites_, target_idx_)] for neighbor_sites_, target_idx_ in zip(neighbor_sites, target_idx)]
        [[bv.octant_analysis() for bv in BVs_] for BVs_ in BVs]
        BVs = [[bv.buried_volume / (bv.buried_volume + bv.free_volume) for bv in BVs_] for BVs_ in BVs]
        des += np.array([[np.max(BVs_), np.min(BVs_)] for BVs_ in BVs]).flatten().tolist()
        charges = np.array(log.read_charge_spin_density()[0])
        des += list(charges[sites])
    else:
        charges = np.array(log.read_charge_spin_density()[0])
        des += [np.max(charges[2]), np.min(charges[sites])]
        if ligand_type == "Ligand_Box": des += [1]
        else: des += [0]
    des += log.read_orbit_eng()
    des += [log.get_dipole()]
    des += [len(sites)]
    qm_dict[ligand_idx] = des

243it [00:53,  4.57it/s]


## Grid Points Descriptors

In [6]:
smiles_csv = pd.read_csv("Data_clear_with_sites.csv")
# smiles_csv = smiles_csv.loc[(smiles_csv['Type'] == "Binol")]
area_dict = {}
for row_id, row in tqdm(smiles_csv.iterrows()):
    ligand_idx = row["Index"]
    conf_id = row["conf_id"]
    ligand_type = row['Type']
    sites = [int(each) for each in row['Sites'].split()]
    mol = Chem.MolFromMolFile(os.path.join(cu_mol_dir, f"{ligand_idx:05}.mol"), removeHs=False)
    if ligand_idx == 1062:
        rwmol = Chem.RWMol(mol)
        rwmol.RemoveAtom(46)
        rwmol.GetAtomWithIdx(15).SetFormalCharge(1)
        mol = rwmol.GetMol()
    log = logfile_process.Logfile(os.path.join(cu_dft_dir, f"{ligand_idx:05}.log"))
    symbol_list, positions = log.symbol_list, log.running_positions[-1]
    cu_atom_id = [atom.GetIdx() for atom in mol.GetAtoms() if atom.GetSymbol() == "Cu"][0]
    out_position = mol_manipulation.trfm_rot(positions[sites[0]], positions[sites[-1]], positions[cu_atom_id], positions)
    if out_position[cu_atom_id][1] < 0:
        out_position = out_position @ mol_manipulation.rotation([1,0,0], 0, -1)
    if ligand_type == "Binol":
        subset = mol.GetSubstructMatches(Chem.MolFromSmarts("[OH]c1ccccc1-c1ccccc1[OH]"))[0]
        assert subset[0] == sites[0] and subset[-1] == sites[-1]
        if out_position[subset[6]][2] < out_position[subset[7]][2]:
            out_position[:, 2] *= -1
    mol = xtb_process.xtb_to_mol(mol, [symbol_list], [out_position], 1)
    sdf_file = os.path.join(cu_mol_dir, f"{ligand_idx:05}.sdf")
    with Chem.SDWriter(sdf_file) as writer:
        writer.write(mol)
    areas = DFThandle.Calc_areas_(mol, sites + [cu_atom_id], radius=8, num_per_axis=20, count_per_axis = [4,4,4])
    out_position = out_position @ mol_manipulation.rotation([1,0,0], 0, -1)
    area_dict[ligand_idx] = areas
    

243it [00:10, 23.27it/s]


In [7]:
with open(f"{data_dir}/all_fp_map2.pkl", 'wb')as f:
    # pickle.dump([rd_mf_map, rd_des_map, morgan_map, modred_map, acsf_3D_map, soap_3D_map, mbtr_3D_map, lmbtr_3D_map, qm_dict, area_dict], f)
    pickle.dump([qm_dict, area_dict], f)

In [ ]:
with open(r"Data/all_fp_map.pkl", 'rb')as f:
    # rd_mf_map, rd_des_map, morgan_map, modred_map, acsf_3D_map, soap_3D_map, mbtr_3D_map, lmbtr_3D_map, qm_dict, area_dict = pickle.load(f)
    qm_dict, area_dict = pickle.load(f)